In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import relation_data_lake as rdl

import re

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()

In [3]:
import umap

2022-12-30 13:25:58.689157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-30 13:25:58.689195: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
salts = ["phosphate", "hydrochloride", "hydrochloride monohydrate", "acetate", "sulfate","disodium",  "sodium"]

def remove_salts(name, salts):
    """
    Remove salt from drug name
    """
    clean_name = name
    for salt in salts:
        try:
            clean_name = re.findall(f"(.+) {salt}", name)[0]
        except:
            continue
    return clean_name

In [5]:
embeddings = pd.read_csv('/home/jupyter-jake/shared_data/collaborations/scripps_oncology/drug_embeddings_after_round_5.csv')
embeddings.set_index('recover_id', inplace=True)
embeddings = embeddings.loc[:,embeddings.std() != 0]

In [6]:
# Get drug info from datalake

drugs = rdl.get_drugs()

recover_drugs = drugs[drugs["relation_id"].isin(embeddings.index)]

recover_drugs

Pulling folder parsed/drugs


,relation_id,name,smiles,mw,max_phase
467,RE-MOL-0000000470,RE-MOL-0000000470,O=c1[nH]c(=O)n(C2CC(O)C(CO)O2)cc1F,246.065200,0
596,RE-MOL-0000000603,RE-MOL-0000000603,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,543.174061,0
796,RE-MOL-0000000818,sirolimus,CO[C@H]1C[C@@H]2CC[C@@H](C)[C@@](O)(O2)C(=O)C(...,913.555142,4
853,RE-MOL-0000000882,clomipramine,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,314.154976,4
907,RE-MOL-0000000938,methoxsalen,COc1c2occc2cc2ccc(=O)oc12,216.042259,4
...,...,...,...,...,...
1369300,RE-MOL-0001376287,spebrutinib,C=CC(=O)Nc1cccc(Nc2nc(Nc3ccc(OCCOC)cc3)ncc2F)c1,423.170668,2
1487964,RE-MOL-0001495681,ribociclib,CN(C)C(=O)c1cc2cnc(Nc3ccc(N4CCNCC4)cn3)nc2n1C1...,434.254258,4
1487970,RE-MOL-0001495720,poziotinib,C=CC(=O)N1CCC(Oc2cc3c(Nc4ccc(Cl)c(Cl)c4F)ncnc3...,490.097474,2
1488032,RE-MOL-0001495947,RE-MOL-0001495947,Cc1ncc(NC(=O)c2ccc(CN3CCN(C)CC3)c(C(F)(F)F)c2)...,562.241642,3


In [7]:
# Some drugs don't have a common name in that table (name is reported as a RE-MOL ID)

no_name = recover_drugs[recover_drugs["name"].str.match("RE-MOL")]

no_name

,relation_id,name,smiles,mw,max_phase
467,RE-MOL-0000000470,RE-MOL-0000000470,O=c1[nH]c(=O)n(C2CC(O)C(CO)O2)cc1F,246.065200,0
596,RE-MOL-0000000603,RE-MOL-0000000603,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,543.174061,0
14128,RE-MOL-0000014222,RE-MOL-0000014222,CC(C)(Oc1ccc(Cl)cc1)C(=O)[O-],213.032395,4
15959,RE-MOL-0000016073,RE-MOL-0000016073,Cc1ccc(-c2cc(C(F)(F)[18F])nn2-c2ccc(S(N)(=O)=O...,380.078417,4
17027,RE-MOL-0000017147,RE-MOL-0000017147,Nc1nc(Cl)nc2c1ncn2C1CC(O)C(CO)O1,285.062867,0
17205,RE-MOL-0000017325,RE-MOL-0000017325,Nc1nc(Cl)nc2c1ncn2C1OC(CO)C(O)C1F,303.053445,0
17317,RE-MOL-0000017437,RE-MOL-0000017437,Nc1nc(F)nc2c1ncn2C1OC(CO)C(O)C1O,285.087332,0
20301,RE-MOL-0000020425,RE-MOL-0000020425,COc1cc([C@@H]2c3cc4c(cc3[C@@H](OC3OC5COC(C)OC5...,588.184291,0
21592,RE-MOL-0000021716,RE-MOL-0000021716,CCCCC(=O)OCC(=O)[C@]1(O)Cc2c(O)c3c(c(O)c2[C@@H...,723.213875,0
28686,RE-MOL-0000028829,RE-MOL-0000028829,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,474.106666,4


In [8]:
f"{len(no_name)} of {len(recover_drugs)} drugs don't have a common name in this table"

"24 of 152 drugs don't have a common name in this table"

In [9]:
# Retrieve extended drug name table
# This table is on the datalake and can include several names for each drug
# I hope to find here some of the "missing" names

all_names = pd.read_csv(rdl.RELATION_DATA_FOLDER + "/parsed/drugs/drug_names.csv")

all_names

,relation_id,name
0,RE-MOL-0000000013,bromoenol lactone
1,RE-MOL-0000000031,ucl-1530
2,RE-MOL-0000000090,zomepirac sodium
3,RE-MOL-0000000092,alpha-bungarotoxin
4,RE-MOL-0000000093,nicotine
...,...,...
121900,RE-MOL-0001724636,edaglitazone
121901,RE-MOL-0001724582,mesuprine
121902,RE-MOL-0001724683,levopropicillin
121903,RE-MOL-0001724584,fostriecin


In [10]:
# Find missing names (keep the first name that appears in extended table)
get_names = no_name.drop(columns="name").merge(all_names, on="relation_id")[["relation_id", "name"]]
unique_names = get_names.groupby("relation_id").first().reset_index()

unique_names

,relation_id,name
0,RE-MOL-0000014222,clofibrate
1,RE-MOL-0000016073,celecoxib
2,RE-MOL-0000028829,vandetanib
3,RE-MOL-0000031117,entinostat
4,RE-MOL-0000334530,theophylline
5,RE-MOL-0000463148,alisertib
6,RE-MOL-0001495947,hhgv-678


Unfortunately, we found only 7 of the missing names.

In [11]:
# Updated name list with the 7 found

update_names = pd.concat([unique_names, recover_drugs[["relation_id", "name"]]]).drop_duplicates(subset=["relation_id"]).reset_index(drop=True)

update_names

,relation_id,name
0,RE-MOL-0000014222,clofibrate
1,RE-MOL-0000016073,celecoxib
2,RE-MOL-0000028829,vandetanib
3,RE-MOL-0000031117,entinostat
4,RE-MOL-0000334530,theophylline
...,...,...
147,RE-MOL-0001319071,gsk-2636771
148,RE-MOL-0001376287,spebrutinib
149,RE-MOL-0001495681,ribociclib
150,RE-MOL-0001495720,poziotinib


In [12]:
update_names

,relation_id,name
0,RE-MOL-0000014222,clofibrate
1,RE-MOL-0000016073,celecoxib
2,RE-MOL-0000028829,vandetanib
3,RE-MOL-0000031117,entinostat
4,RE-MOL-0000334530,theophylline
...,...,...
147,RE-MOL-0001319071,gsk-2636771
148,RE-MOL-0001376287,spebrutinib
149,RE-MOL-0001495681,ribociclib
150,RE-MOL-0001495720,poziotinib


In [13]:
# Experiments
#one_exp = rdl.get_inhouse_data(project='oncology', experiment_round=1, option=2)
#two_exp = rdl.get_inhouse_data(project='oncology', experiment_round=2, option=2)
#three_exp = rdl.get_inhouse_data(project='oncology', experiment_round=3, option=2)
data_folder = '/home/jupyter-jake/shared_data/collaborations/scripps_oncology/'
one_exp = pd.read_csv(data_folder + 'round1_results.csv')
two_exp = pd.read_csv(data_folder + 'round2_results.csv')
three_exp = pd.read_csv(data_folder + 'round3_results.csv')
four_exp = pd.read_csv(data_folder + 'round4_results.csv')
five_exp = pd.read_csv(data_folder + 'round5_results.csv')

In [14]:
# Predictions
one_pred = pd.read_csv(data_folder + 'combos_with_predictions_round1.csv')
two_pred = pd.read_csv(data_folder + 'combos_with_predictions_round2.csv')
three_pred = pd.read_csv(data_folder + 'combos_with_predictions_round3.csv')
four_pred = pd.read_csv(data_folder + 'combos_with_predictions_round4.csv')
five_pred = pd.read_csv(data_folder + 'combos_with_predictions_round5.csv')

columns = ['relation_id_drug_column', 'relation_id_drug_row',
           'preds_mean', 'preds_std', 'score']
columns_one = ['new_drug_id', 'replacement_new_drug_id', 'studied_drug_id',
               'predicted_bliss_max_mean', 'predicted_bliss_max_std',
               'predicted_bliss_max_mean_gnn', 'predicted_bliss_max_std_gnn',
               'predicted_bliss_max_mean_baseline', 'predicted_bliss_max_std_baseline']

one_pred = one_pred[columns_one]
two_pred = two_pred[columns]
three_pred = three_pred[columns]
four_pred = four_pred[columns]
five_pred = five_pred[columns]

In [15]:
# Round 1 (experiment + prediction)
# At this stage we were still using several models including GNN
# Need to confirm with MILA what each prediction means (GNN, Baseline, what about the one without underscore?)

one = one_exp.merge(one_pred, left_on=['drug_col_relation_id', 'drug_row_relation_id'],
                    right_on=['new_drug_id', 'studied_drug_id'])
one_replacements = one_exp.merge(one_pred, left_on=['drug_col_relation_id', 'drug_row_relation_id'],
                             right_on=['replacement_new_drug_id', 'studied_drug_id'])
one = pd.concat([one, one_replacements]).reset_index(drop=True)
one = one[['drug_col_relation_id' ,'drug_row_relation_id',
           'synergy_bliss_max', 'cell_line_name',
           'predicted_bliss_max_mean', 'predicted_bliss_max_std',
           'predicted_bliss_max_mean_gnn', 'predicted_bliss_max_std_gnn',
           'predicted_bliss_max_mean_baseline', 'predicted_bliss_max_std_baseline']]

# Round 2 (experiment + prediction)

two = two_exp.merge(two_pred, left_on=['drug_col_relation_id', 'drug_row_relation_id'],
                    right_on=['relation_id_drug_column', 'relation_id_drug_row'])
two = two[['drug_col_relation_id' ,'drug_row_relation_id',
           'synergy_bliss_max', 'cell_line_name',
           'preds_mean', 'preds_std', 'score']]

# Round 3

three = three_exp.merge(three_pred, left_on=['drug_col_relation_id', 'drug_row_relation_id'],
                    right_on=['relation_id_drug_column', 'relation_id_drug_row'])
three = three[['drug_col_relation_id' ,'drug_row_relation_id',
           'synergy_bliss_max', 'cell_line_name',
           'preds_mean', 'preds_std', 'score']]

# Round 4

four = four_exp.merge(four_pred, left_on=['drug_col_relation_id', 'drug_row_relation_id'],
                    right_on=['relation_id_drug_column', 'relation_id_drug_row'])
four = four[['drug_col_relation_id' ,'drug_row_relation_id',
           'synergy_bliss_max', 'cell_line_name',
           'preds_mean', 'preds_std', 'score']]

# Round 5

five = five_exp.merge(five_pred, left_on=['drug_col_relation_id', 'drug_row_relation_id'],
                    right_on=['relation_id_drug_column', 'relation_id_drug_row'])
five = five[['drug_col_relation_id' ,'drug_row_relation_id',
           'synergy_bliss_max', 'cell_line_name',
           'preds_mean', 'preds_std', 'score']]

# Lookup table for models used in round 1
one_prediction = {'regular' : ['predicted_bliss_max_mean', 'predicted_bliss_max_std'],
                  'gnn': ['predicted_bliss_max_mean_gnn', 'predicted_bliss_max_std_gnn'],
                  'baseline': ['predicted_bliss_max_mean_baseline', 'predicted_bliss_max_std_baseline']}

# Select which model you'd like to consider for round 1 for the remainder of the notebook
# Options: regular, gnn, baseline
# Not sure that's so important since theoretically this is the "random" round
select_one_prediction = 'regular'

base_columns = ['drug_col_relation_id', 'drug_row_relation_id', 'synergy_bliss_max', 'cell_line_name']
prediction_columns = one_prediction[select_one_prediction]
one_select_model = one[base_columns + prediction_columns]
one_select_model = one_select_model.rename(columns={prediction_columns[0]: 'preds_mean',
                          prediction_columns[1]: 'preds_std'})
one_select_model['score'] = one_select_model['preds_mean'] + one_select_model['preds_std']

# Concatenate
one_select_model['round'] = 1
two['round'] = 2
three['round'] = 3
four['round'] = 4
five['round'] = 5
all_rounds = pd.concat([one_select_model, two, three, four, five]).reset_index(drop=True)

all_rounds

,drug_col_relation_id,drug_row_relation_id,synergy_bliss_max,cell_line_name,preds_mean,preds_std,score,round
0,RE-MOL-0001495947,RE-MOL-0000575642,0.0,MCF7,32.175810,3.475047,35.650857,1
1,RE-MOL-0001140916,RE-MOL-0000435288,7.2,MCF7,27.042995,6.442581,33.485576,1
2,RE-MOL-0000584536,RE-MOL-0000006951,18.4,MCF7,19.805845,2.343556,22.149401,1
3,RE-MOL-0001495720,RE-MOL-0000003872,14.2,MCF7,17.392440,1.989863,19.382303,1
4,RE-MOL-0000295701,RE-MOL-0000061053,25.9,MCF7,42.697880,5.554634,48.252514,1
...,...,...,...,...,...,...,...,...
141,RE-MOL-0000031117,RE-MOL-0000042933,7.1,MCF7,17.741364,5.331245,23.072609,5
142,RE-MOL-0001495681,RE-MOL-0000376867,5.5,MCF7,17.372231,5.591420,22.963651,5
143,RE-MOL-0000001992,RE-MOL-0000028829,26.6,MCF7,17.631250,4.403929,22.035180,5
144,RE-MOL-0001518809,RE-MOL-0000042933,22.3,MCF7,16.142559,4.981514,21.124073,5


In [16]:
all_rounds = all_rounds.merge(update_names, left_on="drug_col_relation_id", right_on="relation_id")
all_rounds.rename(columns = {"name":"name_col"})
all_rounds = all_rounds.merge(update_names, left_on="drug_row_relation_id", right_on="relation_id")
all_rounds.rename(columns = {"name":"name_row"})


,drug_col_relation_id,drug_row_relation_id,synergy_bliss_max,cell_line_name,preds_mean,preds_std,score,round,relation_id_x,name_x,relation_id_y,name_y
0,RE-MOL-0001495947,RE-MOL-0000575642,0.0,MCF7,32.175810,3.475047,35.650857,1,RE-MOL-0001495947,hhgv-678,RE-MOL-0000575642,cabazitaxel
1,RE-MOL-0001140916,RE-MOL-0000575642,24.1,MCF7,15.764807,6.448794,22.213601,3,RE-MOL-0001140916,dacomitinib,RE-MOL-0000575642,cabazitaxel
2,RE-MOL-0000584536,RE-MOL-0000575642,1.9,MCF7,15.379181,4.574558,19.953739,3,RE-MOL-0000584536,bms-582949,RE-MOL-0000575642,cabazitaxel
3,RE-MOL-0001376287,RE-MOL-0000575642,10.1,MCF7,13.956803,6.015639,19.972443,3,RE-MOL-0001376287,spebrutinib,RE-MOL-0000575642,cabazitaxel
4,RE-MOL-0000019937,RE-MOL-0000575642,3.6,MCF7,20.723950,7.460007,28.183956,2,RE-MOL-0000019937,terazosin hydrochloride,RE-MOL-0000575642,cabazitaxel
...,...,...,...,...,...,...,...,...,...,...,...,...
141,RE-MOL-0001317309,RE-MOL-0000584456,23.7,MCF7,12.503742,2.832405,15.336147,2,RE-MOL-0001317309,apatinib,RE-MOL-0000584456,vemurafenib
142,RE-MOL-0000001178,RE-MOL-0000584456,12.2,MCF7,14.873702,4.925148,19.798850,3,RE-MOL-0000001178,trifluoperazine hydrochloride,RE-MOL-0000584456,vemurafenib
143,RE-MOL-0001495681,RE-MOL-0000408487,25.4,MCF7,16.275003,4.709943,20.984946,5,RE-MOL-0001495681,ribociclib,RE-MOL-0000408487,vismodegib
144,RE-MOL-0000401940,RE-MOL-0000408487,-1.9,MCF7,11.486040,2.463049,13.949089,2,RE-MOL-0000401940,pd-0325901,RE-MOL-0000408487,vismodegib


In [17]:
for ii in range(len(all_rounds)):
    all_rounds.name_x[ii] = remove_salts(all_rounds.name_x[ii], salts)
    all_rounds.name_y[ii] = remove_salts(all_rounds.name_y[ii], salts)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
all_rounds.name_y[all_rounds.name_y == "[11c]-sorafenib"] = "sorafenib"

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
all_rounds.name_x[all_rounds.name_x == "[11c]epinephrine"] = "epinephrine"

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
all_rounds['round'][all_rounds['round'] == 1] = 'Calibration'
all_rounds['round'][all_rounds['round'] == 2] = 'Diversity'
all_rounds['round'][all_rounds['round'] == 3] = 'SMO Round 1'
all_rounds['round'][all_rounds['round'] == 4] = 'SMO Round 2'
all_rounds['round'][all_rounds['round'] == 5] = 'SMO Round 3'

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so w

In [21]:
all_rounds[['name_x','name_y']].duplicated().sum()

0

In [64]:
all_rounds.to_csv('/home/jupyter-jake/shared_notebooks/collaborations/recover/scripps_oncology/Sankey_output.csv')

In [30]:
len(pd.concat([all_rounds.drug_col_relation_id, all_rounds.drug_row_relation_id]).unique())

91

In [ ]:
len(pd.concat([all_rounds.drug_col_relation_id, all_rounds.drug_row_relation_id]).unique())

In [35]:
tmp = all_rounds[(all_rounds['round'] == 'Calibration')|(all_rounds['round'] == 'Diversity')]
tmp.shape

(58, 12)

In [36]:
len(pd.concat([tmp.drug_col_relation_id, tmp.drug_row_relation_id]).unique())

89